In [1]:
import pymorphy2
import re
import nltk

In [2]:
morph = pymorphy2.MorphAnalyzer()
lemmatizer = nltk.stem.WordNetLemmatizer() 

In [3]:
sentence_rus = 'Программа направлена на получение гуманитарного и математического образования, '\
                'а также навыков программирования.'

sentence_eng = "This course is aimed at building students’ skills in the humanities and in mathematics, "\
                "as well as in programming."

In [4]:
POSes = {'ADJ': ['ADJ', 'ADJS', 'ADJF', 'PRED'], 'NOUN': 'NOUN', 'VERB': ['VERB', 'INFN', 'GRND', 'PRTF', 'PRTS'],
       'PRT': ['PRT', 'PREP'], 'PRON': ['PRON', 'NPRO'], 'ADV': ['ADV', 'ADVB', 'COMP'], 'NUM': ['NUM', 'NUMR'],
       'CONJ': 'CONJ', 'ADP': ['ADP', 'PREP'], 'PRT': ['PRT', 'PRCL', 'INTJ'], 'DET': 'DET'}

In [5]:
class Sentence:   # полиморфизм реализован в том, что сперва создается класс Sentence, определяющий набор методов
                  # с предложением, а уже после классы RussianSentence и EnglishSentence доопределяют заданные методы
    '''
    Класс Sentence является базовым классом для работы с предложением, специфика работы с русским и английским
    предложением отражена в классах потомках RussianSentence и EnglishSentence
    '''
    
    def __init__(self, sentence):
        '''
        Конструктор
        '''
        self.sentence = sentence
        
    def get_words(self):
        '''
        Получить список слов
        '''
        return self.words    # так можно делать? возвращать до того, как определяю (я определяю self.words в 
                             # каждом из классов по-разному)
    
    def get_lemmas(self):
        '''
        Получить список лемм
        '''
        pass
    
    def get_pos(self, pos):
        '''
        Получить список слов с заданной частью речи
        '''
        pass
                
    def check_affirm(self):
        '''
        Определяет тип предложения
        '''
        if self.sentence.endswith('?'):
            print('Вопросительное')
        elif self.sentence.endswith('!'):
            print('Восклицательное')
        else:
            print('Удтвердительное')
    
    def __del__(self):
        '''
        Деструктор
        '''
        print('Вызван деструктор, экземпляр удален!')

In [6]:
class RussianSentence(Sentence):
    
    def __init__(self, sentence):
        '''
        Конструктор
        '''
        super().__init__(sentence=sentence)
        self.words = pymorphy2.tokenizers.simple_word_tokenize(self.sentence)
    
    def get_lemmas(self):
        '''
        Получить список лемм
        '''
        lemmas = []
        for word in self.words:
            lemmas.append(morph.parse(word)[0].lexeme[0][0])
        return lemmas
    
    def get_pos(self, pos):
        '''
        Получить список слов с заданной частью речи, можно найти по : ADJ,
        NOUN, VERB, PRT, PRON, ADV, NUM, CONJ, ADP, DET.
        '''
        try:
            word_list = []
            for word in self.words:
                if str(morph.parse(word)[0].tag.POS) in POSes[pos]:
                    word_list.append(word)
            if not word_list:
                print('В предложении слов заданной части речи нет')
            else:
                return word_list
        except KeyError:
            print('Неправильно задана часть речи, можно проверить по: ADJ, '\
                  'NOUN, VERB, PRT, PRON, ADV, NUM, CONJ, ADP, DET.')

In [7]:
class EnglishSentence(Sentence):
    
    def __init__(self, sentence):
        '''
        Конструктор
        '''
        super().__init__(sentence=sentence)
        self.words = nltk.word_tokenize(self.sentence)
        
    def get_lemmas(self):
        '''
        Получить список лемм
        '''
        lemmas = []
        for word in self.words:
            lemmas.append(lemmatizer.lemmatize(word))
        return lemmas
    
    def get_pos(self, pos):
        '''
        Получить список слов с заданной частью речи, можно найти по : ADJ,
        NOUN, VERB, PRT, PRON, ADV, NUM, CONJ, ADP, DET.
        '''
        try:
            word_list = []
            posTagged = nltk.pos_tag(self.words)
            simplifiedTags = [(word, nltk.map_tag('en-ptb', 'universal', tag)) for word, tag in posTagged]
            for word in simplifiedTags:
                if word[1] in POSes[pos]:
                    word_list.append(word[0])
            if not word_list:
                print('В предложении слов заданной части речи нет')
            else:
                return word_list
        except KeyError:
            print('Неправильно задана часть речи, можно найти по: ADJ, '\
                  'NOUN, VERB, PRT, PRON, ADV, NUM, CONJ, ADP, DET.')